# ByteNite Guide: Local Folder Batch Upload
Batch encoding - Local upload feature

---
The aim of this notebook is to **batch encode all the video contents in a folder** located on this computer using ByteNite Encoding API.

_Note: make sure you have enough ByteChips in your wallet: https://app.bytenite.com/wallet (In this demo, every input minute costs ~20 ByteChips)._


<h1 style="color:blue">Setup (edit with your info)</h1>

## Your API Key ↓
Get it at https://app.bytenite.com/profile

In [7]:
# >> Paste here your API key (Go to app.bytenite.com/profile)
access_token = 'BYTENITE_ACCESS_TOKEN'

## Your local video folder ↓

In [8]:
# >> Please specify the location of the folder containing the videos to be transcoded
my_dir = "YOUR_FOLDER_PATH"

<h1 style="color:orange">Additional setup (edit if you need different encoding options)</h1>
The default encoding parameters for this notebook are:

- H.265 540p @600k
- H.265 1080p @4.5M

You can configure your parameters from the "New Job" wizard on app.bytenite.com, and then copy-paste the document by clicking on "Switch to JSON" on the top right.

_Note: "true" / "false" values must be changed to Python's "True" / "False" (capital)_

In [9]:
## Job parameters dictionary (JSON)
job_params_body={
"data":{"output_code_template": "",
        "output_template": "{{.job_name}}_{{.output_params.Aspect.Resolution.Height}}",
        "outputs":[{"output_params":{"aspect":{"cropping":{"bottom":0,"left":0,"right":0,"top":0},"orientation":{"flip":"","rotation":""},"padding":{"fill_color":"#000000","final_aspect_ratio":""},"resolution":{"aspect_ratio":"","avoid_upscaling":True,"height":"540p"}},"audio":{"audio_codec":"AAC","audio_channel":"2.0","audio_sample_rate":"48khz","audio_bitrate":"192k"},"video":{"bitrate":{"rate_control_mode":"constant_bitrate","target_bitrate":"600k"},"codec_params":{"preset":"medium","profile":"","tune":""},"frame_rate":{"fps":"","up_mode":"avoid"},"codec":"libx265"}},"output_type":"mp4"},
                    {"output_params":{"aspect":{"cropping":{"bottom":0,"left":0,"right":0,"top":0},"orientation":{"flip":"","rotation":""},"padding":{"fill_color":"#000000","final_aspect_ratio":""},"resolution":{"aspect_ratio":"","avoid_upscaling":True,"height":"1080p"}},"audio":{"audio_codec":"AAC","audio_channel":"2.0","audio_sample_rate":"48khz","audio_bitrate":"192k"},"video":{"bitrate":{"rate_control_mode":"constant_bitrate","target_bitrate":"4500k"},"codec_params":{"preset":"medium","profile":"","tune":""},"frame_rate":{"fps":"","up_mode":"avoid"},"codec":"libx265"}},"output_type":"mp4"}
                   ]},"preset":"unpackaged"
}



-------



<h1 style="color:red">Execution (do not alter)</h1>

In [10]:
# Imports
import random
import string
import requests
import os
import time

<h2 style="color:red">Check your input files</h2>

In [11]:
# Get all the filenames of videos in the folder
def is_video_file(filename):
  video_file_extensions = ('.264', '.3g2', '.3gp', '.3gp2', '.3gpp', '.3gpp2', '.3mm', '.3p2', '.60d', '.787', '.89', '.aaf', '.aec', '.aep', '.aepx', '.aet', '.aetx', '.ajp', '.ale', '.am', '.amc', '.amv', '.amx', '.anim', '.aqt', '.arcut', '.arf', '.asf', '.asx', '.avb', '.avc', '.avd', '.avi', '.avp', '.avs', '.avs', '.avv', '.axm', '.bdm', '.bdmv', '.bdt2', '.bdt3', '.bik', '.bin', '.bix', '.bmk', '.bnp', '.box', '.bs4', '.bsf', '.bvr', '.byu', '.camproj', '.camrec', '.camv', '.ced', '.cel', '.cine', '.cip', '.clpi', '.cmmp', '.cmmtpl', '.cmproj', '.cmrec', '.cpi', '.cst', '.cvc', '.cx3', '.d2v', '.d3v', '.dat', '.dav', '.dce', '.dck', '.dcr', '.dcr', '.ddat', '.dif', '.dir', '.divx', '.dlx', '.dmb', '.dmsd', '.dmsd3d', '.dmsm', '.dmsm3d', '.dmss', '.dmx', '.dnc', '.dpa', '.dpg', '.dream', '.dsy', '.dv', '.dv-avi', '.dv4', '.dvdmedia', '.dvr', '.dvr-ms', '.dvx', '.dxr', '.dzm', '.dzp', '.dzt', '.edl', '.evo', '.eye', '.ezt', '.f4p', '.f4v', '.fbr', '.fbr', '.fbz', '.fcp', '.fcproject', '.ffd', '.flc', '.flh', '.fli', '.flv', '.flx', '.gfp', '.gl', '.gom', '.grasp', '.gts', '.gvi', '.gvp', '.h264', '.hdmov', '.hkm', '.ifo', '.imovieproj', '.imovieproject', '.ircp', '.irf', '.ism', '.ismc', '.ismv', '.iva', '.ivf', '.ivr', '.ivs', '.izz', '.izzy', '.jss', '.jts', '.jtv', '.k3g', '.kmv', '.ktn', '.lrec', '.lsf', '.lsx', '.m15', '.m1pg', '.m1v', '.m21', '.m21', '.m2a', '.m2p', '.m2t', '.m2ts', '.m2v', '.m4e', '.m4u', '.m4v', '.m75', '.mani', '.meta', '.mgv', '.mj2', '.mjp', '.mjpg', '.mk3d', '.mkv', '.mmv', '.mnv', '.mob', '.mod', '.modd', '.moff', '.moi', '.moov', '.mov', '.movie', '.mp21', '.mp21', '.mp2v', '.mp4', '.mp4v', '.mpe', '.mpeg', '.mpeg1', '.mpeg4', '.mpf', '.mpg', '.mpg2', '.mpgindex', '.mpl', '.mpl', '.mpls', '.mpsub', '.mpv', '.mpv2', '.mqv', '.msdvd', '.mse', '.msh', '.mswmm', '.mts', '.mtv', '.mvb', '.mvc', '.mvd', '.mve', '.mvex', '.mvp', '.mvp', '.mvy', '.mxf', '.mxv', '.mys', '.ncor', '.nsv', '.nut', '.nuv', '.nvc', '.ogm', '.ogv', '.ogx', '.osp', '.otrkey', '.pac', '.par', '.pds', '.pgi', '.photoshow', '.piv', '.pjs', '.playlist', '.plproj', '.pmf', '.pmv', '.pns', '.ppj', '.prel', '.pro', '.prproj', '.prtl', '.psb', '.psh', '.pssd', '.pva', '.pvr', '.pxv', '.qt', '.qtch', '.qtindex', '.qtl', '.qtm', '.qtz', '.r3d', '.rcd', '.rcproject', '.rdb', '.rec', '.rm', '.rmd', '.rmd', '.rmp', '.rms', '.rmv', '.rmvb', '.roq', '.rp', '.rsx', '.rts', '.rts', '.rum', '.rv', '.rvid', '.rvl', '.sbk', '.sbt', '.scc', '.scm', '.scm', '.scn', '.screenflow', '.sec', '.sedprj', '.seq', '.sfd', '.sfvidcap', '.siv', '.smi', '.smi', '.smil', '.smk', '.sml', '.smv', '.spl', '.sqz', '.srt', '.ssf', '.ssm', '.stl', '.str', '.stx', '.svi', '.swf', '.swi', '.swt', '.tda3mt', '.tdx', '.thp', '.tivo', '.tix', '.tod', '.tp', '.tp0', '.tpd', '.tpr', '.trp', '.ts', '.tsp', '.ttxt', '.tvs', '.usf', '.usm', '.vc1', '.vcpf', '.vcr', '.vcv', '.vdo', '.vdr', '.vdx', '.veg','.vem', '.vep', '.vf', '.vft', '.vfw', '.vfz', '.vgz', '.vid', '.video', '.viewlet', '.viv', '.vivo', '.vlab', '.vob', '.vp3', '.vp6', '.vp7', '.vpj', '.vro', '.vs4', '.vse', '.vsp', '.w32', '.wcp', '.webm', '.wlmp', '.wm', '.wmd', '.wmmp', '.wmv', '.wmx', '.wot', '.wp3', '.wpl', '.wtv', '.wve', '.wvx', '.xej', '.xel', '.xesc', '.xfl', '.xlmv', '.xmv', '.xvid', '.y4m', '.yog', '.yuv', '.zeg', '.zm1', '.zm2', '.zm3', '.zmv'  ) 
  if filename.endswith((video_file_extensions)):
    return True
  return False
obj = os.scandir(my_dir)
video_filenames = [entry.name for entry in obj if entry.is_file() and is_video_file(entry.name)]
print("Video files in '% s':" % my_dir)
print(video_filenames)

video_filenames.sort()
# Prepare the job names and filepaths
job_names = [s[:min(s.rfind("."),25)] for s in video_filenames]
source_filepaths = [os.path.join(my_dir, v) for v in video_filenames]

Video files in '/Users/fabio/transcoding/test/':
['A001.mp4', 'A003.mp4', 'A002.mp4']


<h2 style="color:red">Jobs creation and submission via ByteNite API</h2>

In [12]:
# JOBS CREATION

## Data source and data destination definition
data_source_body={
"dataSource": {"dataSourceDescriptor":"file", "params":{"@type":"type.googleapis.com/bytenite.data_source.LocalFileDataSource","name":"","url":"","password":"","username":""}},
"dataDestination": {"dataSourceDescriptor":"bucket", "params":{}}
}

# BATCH JOB CREATION
n = min(len(source_filepaths), 100)
job_ids = ['']*n
job_data = ['']*n

## For error log
create_resp = ['']*n
datasource_resp = ['']*n
upload_resp = ['']*n
params_resp = ['']*n
run_resp = ['']*n
temp_url = ['']*n

for i in range(n):
  
    # CREATE JOB
    create_job_body = {"name": job_names[i], "templateId": "video-transcoding@v0.2"}
    create_resp[i] = requests.post('http://api.bytenite.com/v1/customer/jobs', json=create_job_body, headers={'Authorization': access_token}).json()
    try:
        job_data[i] = create_resp[i]['job']
        job_ids[i] = job_data[i]['id']
        print(f"Job #{i} ({job_names[i]}) created - ID: {job_ids[i]}")
    except:
        print(f"CREATE ERROR: Job #{i} ({job_names[i]}):")
        print(create_resp[i])
               
    # SET SOURCE AND DESTINATION
    datasource_resp[i] = requests.post(f'https://api.bytenite.com/v1/customer/jobs/datasource/{job_ids[i]}', json=data_source_body, headers={'Authorization': access_token}).json()
    try:
        job_data[i] = datasource_resp[i]['job']
        temp_url[i] = job_data[i]['dataSource']['params']['tempUrl']
    except:
        print(f"DATASOURCE ERROR: Job #{i} ({job_names[i]}):")
        print(datasource_resp[i])

    # UPLOAD FILE
    try:
        print("Uploading...")
        with open(source_filepaths[i], mode='rb') as f:
            files = {video_filenames[i]: f}
            upload_resp[i] = requests.put(temp_url[i], data=f, headers={'Content-Disposition': f'attachment; filename="{source_filepaths[i]}"'}) #requests.put(temp_url[i], files=files)
            upload_resp[i].raise_for_status()
            confirm_upload_resp = requests.post(f'http://api.bytenite.com/v1/customer/jobs/uploadcompleted/{job_ids[i]}', 
                                    json={}, 
                                    headers={'Authorization': access_token})
            confirm_upload_resp.raise_for_status()
    except:
        print(f"UPLOAD ERROR: Job #{i} ({job_names[i]}):")
        print(upload_resp[i])
    
    # SET PARAMETERS
    params_resp[i] = requests.post(f'https://api.bytenite.com/v1/customer/jobs/params/{job_ids[i]}', json=job_params_body, headers={'Authorization': access_token}).json()
    try:
        job_data[i] = params_resp[i]['job']
    except:
        print(f"PARAMETERS SETTING ERROR: Job #{i} ({job_names[i]}):")
        print(params_resp[i])
        
    time.sleep(0.5)
    
    # RUN JOB
    run_resp[i] = requests.post(f'https://api.bytenite.com/v1/customer/jobs/run/{job_ids[i]}', json={}, headers={'Authorization': access_token}).json()
    try:
        job_started = run_resp[i]['ok']
        print("Job started!\n-------------------------\n")
        
    except:
        print(f"RUN JOB ERROR: Job #{i} ({job_names[i]}):")
        print(run_resp[i])
        print("-------------------------\n")


Job #0 (A001) created - ID: a371c3fe-1ebd-4799-bc62-7269294bc30c
Uploading...
Job started!
-------------------------

Job #1 (A002) created - ID: 37866e2e-c924-4541-8143-eb9be0c40c30
Uploading...
Job started!
-------------------------

Job #2 (A003) created - ID: c5f67687-3e53-4b50-9624-cb22101cbb14
Uploading...
Job started!
-------------------------



-------


<h1 style="color:green">Get the results (run after launching the jobs)</h1>

In [7]:
import dateutil.parser
from IPython.display import display, HTML

results_resp = ['']*n
results_data = ['']*n
job_time = ['']*n


for i in range(len(job_ids)):
    results_resp[i] = requests.get(f'https://api.bytenite.com/v1/customer/jobs/{job_ids[i]}/results', headers={'Authorization': access_token}).json()
    print(f"{job_names[i]} - Outputs")
    try:
        results_data[i] = results_resp[i]['results']
        job_data[i] = requests.get(f'https://api.bytenite.com/v1/customer/jobs/{job_ids[i]}', json=job_params_body, headers={'Authorization': access_token}).json()['job']
        job_time[i] = (dateutil.parser.isoparse(job_data[i]['completedAt']) - dateutil.parser.isoparse(job_data[i]['startedAt'])).total_seconds()
        print(f"Running time: {int(job_time[i])} seconds")
        for j in range(max(len(results_data[i]),1)):
            display(HTML(f"<a href='{results_data[i][j]['link']}' target='_blank'>Link {j+1}</a>"))
    except KeyError:
        print(f"ERROR: {results_resp[i]}")
    except (IndexError, TypeError):
        display(HTML(f"<b>Not ready</b> (wait and rerun cell)..."))
    print("\n----------------------------\n")

A001 - Outputs



----------------------------

A002 - Outputs



----------------------------

A003 - Outputs



----------------------------



## Troubleshooting

For more info, visit https://github.com/ByteNite2/bytenite-api-docs or contact us at support@bytenite.com